In [293]:
import numpy as np
import pandas as pd
import requests
import json
import re
from functools import reduce

In [2]:
import dateutil
from dateutil.parser import parse
import datetime

getting weather *measurements* datapoints (for solar irradiance) from NREL SRRL BMS through the API:<br>
https://internal-apis.nrel.gov/intelligentcampus/hisRead?id=@p:nrel:r:20ed5df2-2c0e126b&range="2018-08-31%2c2018-09-02"

In the above request, note the manner in which the date range is specified.

to plug in the ID of the specific measurement (Dry Bulb Temperature in the following example) in the above link:<br>
NREL => Historian => Weather => NREL SRRL BMS => Dry Bulb Temperature => Aspects => Folio => ID

List of ids:
    - Energy Consumption => unit kwh => @p:nrel:r:225918db-bfbda16a
    - Relative Humidity (RH) => unit %RH  => @p:nrel:r:20ed5e0a-275dbdc2
    - Barometric Pressure (BP) => unit mbar  => @p:nrel:r:20ed5e0a-53e174aa
    - Dry Bulb Temperature (DBT) => unit degree Far => @p:nrel:r:20ed5e0a-fe755c80
    - Global Horizontal Irradiance (GHI) => unit watss/m2_irr => @p:nrel:r:20ed5df2-2c0e126b
    - Total Cloud Cover(TCC) => unit % => @p:nrel:r:20ed5e0a-acc8beff
    - Wind Speed at 19’ (5.7 meter) (WS) => unit mph => @p:nrel:r:20ed5df2-fd2eecc5
    

In [98]:
start_date = '2018-10-22'
end_date = '2018-10-26'
start_time = '00:01:00'
end_time = '23:59:00'

### requesting the data from API
- storing it in a dictionary

In [4]:
root_url = 'https://internal-apis.nrel.gov/intelligentcampus/hisRead?id='
reference_id = ['@p:nrel:r:225918db-bfbda16a','@p:nrel:r:20ed5e0a-275dbdc2','@p:nrel:r:20ed5e0a-53e174aa',
                '@p:nrel:r:20ed5e0a-fe755c80','@p:nrel:r:20ed5df2-2c0e126b','@p:nrel:r:20ed5e0a-acc8beff',
                '@p:nrel:r:20ed5df2-fd2eecc5']
date_range = '&range=\"'+start_date+'%2c'+end_date+'\"'
feat_name = ['EC','RH','BP','DBT','GHI','TCC','WS']

response_dict = {}
for i in range(len(reference_id)):
    response_dict['resp_'+feat_name[i]] = requests.get(root_url+reference_id[i]+date_range)
    if response_dict['resp_'+feat_name[i]].status_code == 200:
        pass
    else:
        print("response from {} is not getting fetched from API".format(feat_name[i]))

In [5]:
EC = response_dict['resp_EC'].content.decode('utf-8').split("\n")
EC = EC[2:]
EC = filter(None, EC)
len(list(EC))

480

In [6]:
def str_split(row):
    time_val = row.split(",")[0].strip(" Denver")
    energy_val = row.split(",")[1]
    energy_val = re.sub('[kwh%RHmbar°FW/m²_irrp]','', energy_val)
    return (time_val, float(energy_val))

def date_parser(row):
    parsed = parse(row)
    datetime_var = parsed.strftime(format='%m-%d-%y %H:%M:%S')
    date = parsed.date()
    time = parsed.time()
    return (datetime_var ,date, time)

### parsing the data (which came as string of non-json format) to get the timestamp and feat_value
- implemented to scale for all the input features
- str_split function used for splitting datetime and feat_values
- date_parser function used for converting the datetime into desired datetime strings

In [7]:
feat_name = ['EC','RH','BP','DBT','GHI','TCC','WS']
parsed_dict = {}
for i in range(len(feat_name)):
    parsed_dict[feat_name[i]] = response_dict['resp_'+feat_name[i]].content.decode('utf-8').split("\n")
    parsed_dict[feat_name[i]] = parsed_dict[feat_name[i]][2:]
    parsed_dict[feat_name[i]] = filter(None, parsed_dict[feat_name[i]])
    parsed_dict[feat_name[i]] = list(map(str_split, parsed_dict[feat_name[i]]))
    
    # the following line gives list (len 2) of lists (i.e. EC_dt and EC_value)
    # i.e. EC_dt, EC_value = EC[0], EC[1]
    parsed_dict[feat_name[i]] = list(zip(*parsed_dict[feat_name[i]]))
    
    # parsing the datetimeinfo obtained in above list into datetime string, date and time
    # the lists can be unpacked as:
    # EC_datetime, EC_date, EC_time = EC_dt_parsed[0], EC_dt_parsed[1], EC_dt_parsed[2]
    parsed_dict[feat_name[i]+'_dt_parsed'] = list(map(date_parser, parsed_dict[feat_name[i]][0]))
    parsed_dict[feat_name[i]+'_dt_parsed'] = list(zip(*parsed_dict[feat_name[i]+'_dt_parsed']))

In [8]:
parsed_dict.keys()

dict_keys(['BP', 'WS', 'DBT_dt_parsed', 'RH_dt_parsed', 'RH', 'EC_dt_parsed', 'BP_dt_parsed', 'TCC_dt_parsed', 'EC', 'TCC', 'DBT', 'GHI', 'GHI_dt_parsed', 'WS_dt_parsed'])

### sample feature (GHI) data pre-processing
- converting the parsed lists into dataframe
- inserting the intended first and the last time-stamp (if not present already)
- setting the index as DatetimeIndex to fill-in the missing time stamps
- filling in the nan values for the feature with the average of before and after value

In [262]:
df_GHI = pd.DataFrame({'datetime_str':parsed_dict['GHI_dt_parsed'][0],'GHI':parsed_dict['GHI'][1],},
                 columns=['datetime_str','GHI'])
print("shape of raw dataframe: {}".format(df_GHI.shape))

df_GHI['datetime_str'] = pd.to_datetime(df_GHI['datetime_str'])
if not (df_GHI.loc[0, 'datetime_str'] == pd.to_datetime(start_date+' '+start_time)):
    df_GHI.loc[0, 'datetime_str'] = pd.to_datetime(start_date+' '+start_time)
if not (df_GHI.loc[df_GHI.index[-1], 'datetime_str'] == pd.to_datetime(end_date+' '+end_time)):
    df_GHI.loc[df_GHI.index[-1], 'datetime_str'] = pd.to_datetime(end_date+' '+end_time)
    
df_GHI=df_GHI.set_index('datetime_str').resample("1min").first().reset_index().reindex(columns=df_GHI.columns)
cols = df_GHI.columns.difference(['GHI'])
df_GHI[cols] = df_GHI[cols].ffill()
df_GHI['GHI'] = df_GHI['GHI'].fillna(((df_GHI['GHI'].shift() + df_GHI['GHI'].shift(-1))/2))
print("shape of processed dataframe: {}".format(df_GHI.shape))

shape of raw dataframe: (7170, 2)
shape of processed dataframe: (7199, 2)


### scaling the above sample for all the input features (excluding the target feature :EC)
- using a dictionary to store the individual dataframes for the input features

In [269]:
input_feat_name = ['RH','BP','DBT','GHI','TCC','WS']
df_dict = {}
for i in range(len(input_feat_name)):
    df_dict["df_"+input_feat_name[i]] = pd.DataFrame({'datetime_str':parsed_dict[input_feat_name[i]+'_dt_parsed'][0],
                                                input_feat_name[i]:parsed_dict[input_feat_name[i]][1]},
                                               columns=['datetime_str',input_feat_name[i]])
    
    df_temp = df_dict["df_"+input_feat_name[i]]
    df_temp.name = "df_"+input_feat_name[i]
    print("raw_dataframe = {}, shape = {}".format(df_temp.name,df_temp.shape))
    df_temp['datetime_str'] = pd.to_datetime(df_temp['datetime_str'])

    if not (df_temp.loc[0, 'datetime_str'] == pd.to_datetime(start_date+' '+start_time)):
        df_temp.loc[0, 'datetime_str'] = pd.to_datetime(start_date+' '+start_time)
    if not (df_temp.loc[df_temp.index[-1], 'datetime_str'] == pd.to_datetime(end_date+' '+end_time)):
        df_temp.loc[df_temp.index[-1], 'datetime_str'] = pd.to_datetime(end_date+' '+end_time)
    
    df_temp=df_temp.set_index('datetime_str').resample("1min").first().reset_index().reindex(columns=df_temp.columns)
    cols = df_temp.columns.difference([input_feat_name[i]])
    df_temp[cols] = df_temp[cols].ffill()
    df_temp[input_feat_name[i]] = df_temp[input_feat_name[i]].fillna(((df_temp[input_feat_name[i]].shift() + df_temp[input_feat_name[i]].shift(-1))/2))
    print("shape of processed dataframe: {}".format(df_temp.shape))

    df_dict["df_"+input_feat_name[i]] = df_temp
    del df_temp

raw_dataframe = df_RH, shape = (7171, 2)
shape of processed dataframe: (7199, 2)
raw_dataframe = df_BP, shape = (7171, 2)
shape of processed dataframe: (7199, 2)
raw_dataframe = df_DBT, shape = (7171, 2)
shape of processed dataframe: (7199, 2)
raw_dataframe = df_GHI, shape = (7170, 2)
shape of processed dataframe: (7199, 2)
raw_dataframe = df_TCC, shape = (7171, 2)
shape of processed dataframe: (7199, 2)
raw_dataframe = df_WS, shape = (7170, 2)
shape of processed dataframe: (7199, 2)


### Processing the EC raw values separately

In [270]:
df_EC = pd.DataFrame({'datetime_str':parsed_dict['EC_dt_parsed'][0],'EC':parsed_dict['EC'][1],},
                 columns=['datetime_str','EC'])

In [278]:
df_EC.tail()

,datetime_str,EC
475,10-26-18 22:45:00,1180.369629
476,10-26-18 23:00:00,1174.185425
477,10-26-18 23:15:00,1179.348145
478,10-26-18 23:30:00,1180.798950
479,10-26-18 23:45:00,1172.009888


In [279]:
EC_start_time = '00:00:00'
EC_end_time = '23:45:00'

In [281]:
df_EC = pd.DataFrame({'datetime_str':parsed_dict['EC_dt_parsed'][0],'EC':parsed_dict['EC'][1],},
                 columns=['datetime_str','EC'])
print("shape of raw dataframe: {}".format(df_EC.shape))

df_EC['datetime_str'] = pd.to_datetime(df_EC['datetime_str'])
if not (df_EC.loc[0, 'datetime_str'] == pd.to_datetime(start_date+' '+EC_start_time)):
    df_EC.loc[0, 'datetime_str'] = pd.to_datetime(start_date+' '+EC_start_time)
if not (df_EC.loc[df_EC.index[-1], 'datetime_str'] == pd.to_datetime(end_date+' '+EC_end_time)):
    df_EC.loc[df_EC.index[-1], 'datetime_str'] = pd.to_datetime(end_date+' '+EC_end_time)
    
df_EC=df_EC.set_index('datetime_str').resample("15min").first().reset_index().reindex(columns=df_EC.columns)
cols = df_EC.columns.difference(['EC'])
df_EC[cols] = df_EC[cols].ffill()
df_EC['EC'] = df_EC['EC'].fillna(((df_EC['EC'].shift() + df_EC['EC'].shift(-1))/2))
print("shape of processed dataframe: {}".format(df_EC.shape))

shape of raw dataframe: (480, 2)
shape of processed dataframe: (480, 2)


In [312]:
df_EC.head()

,datetime_str,EC
0,2018-10-22 00:00:00,1121.757446
1,2018-10-22 00:15:00,1124.623535
2,2018-10-22 00:30:00,1137.146484
3,2018-10-22 00:45:00,1127.491821
4,2018-10-22 01:00:00,1131.487305


### Merging the dataframes of the input features
- reference (https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes/44338256)

In [306]:
df_list = []
for key, value in df_dict.items():
    df_list.append(df_dict[key])
    
input_df = reduce(lambda left,right: pd.merge(left,right,on=['datetime_str'], how='outer'), df_list)

In [311]:
input_df.head()

,datetime_str,RH,DBT,TCC,GHI,BP,WS
0,2018-10-22 00:01:00,32.209999,54.698002,-1.0,-1.147713,819.689575,8.556530
1,2018-10-22 00:02:00,32.244999,54.698002,-1.0,-1.173408,819.702087,8.388755
2,2018-10-22 00:03:00,32.279999,54.698002,-1.0,-1.199104,819.714600,8.220980
3,2018-10-22 00:04:00,32.209999,54.698002,-1.0,-1.239078,819.728577,6.655079
4,2018-10-22 00:05:00,32.980000,54.644001,-1.0,-1.235390,819.731812,5.733435
5,2018-10-22 00:06:00,34.169998,54.301998,-1.0,-1.257245,819.725830,6.740085
6,2018-10-22 00:07:00,34.660000,53.762001,-1.0,-1.258151,819.724670,8.668380
7,2018-10-22 00:08:00,34.880001,53.419998,-1.0,-1.197758,819.690308,9.645949
8,2018-10-22 00:09:00,34.880001,53.203999,-1.0,-1.126652,819.666870,10.093350
9,2018-10-22 00:10:00,35.290001,53.113998,-1.0,-1.074014,819.674805,9.590025


In [315]:
input_df =input_df.set_index('datetime_str').resample("15min").first().reset_index().reindex(columns=input_df.columns)

In [318]:
input_df.head()

,datetime_str,RH,DBT,TCC,GHI,BP,WS
0,2018-10-22 00:00:00,32.209999,54.698002,-1.0,-1.147713,819.689575,8.556530
1,2018-10-22 00:15:00,34.990002,52.897999,-1.0,-0.993266,819.652283,10.764450
2,2018-10-22 00:30:00,34.139999,53.456001,-1.0,-1.161014,819.693481,8.641536
3,2018-10-22 00:45:00,34.990002,52.717999,-1.0,-1.145272,819.653870,10.402057
4,2018-10-22 01:00:00,33.970001,53.096001,-1.0,-1.167657,819.607605,11.211851


#### To do:
- normalize input_df and df_EC
- divide into train and test
- make the dataset iterable
- convert in into torch tensors
- build the 